# CONFLUENCE Tutorial: Point-Scale Workflow (Flux Tower Example)

This notebook demonstrates CONFLUENCE's simplest spatial configuration: point-scale modeling. We'll simulate vertical processes at a single location, typical for flux tower sites, weather stations, or snow monitoring locations.

## Overview

Point-scale modeling focuses on vertical processes without spatial heterogeneity:
- Energy balance
- Soil moisture dynamics  
- Snow accumulation and melt
- Evapotranspiration

## Learning Objectives

1. Understand point-scale modeling in CONFLUENCE
2. Configure CONFLUENCE for single-point simulations
3. Compare with spatially distributed approaches
4. Analyze point-scale model outputs

## 1. Understanding Point-Scale Modeling

Point-scale modeling is the foundation of land surface modeling. Before we simulate entire watersheds, we need to understand the vertical processes at a single point.

### When to Use Point-Scale Modeling:

- **Flux tower validation**: Compare model outputs with eddy covariance measurements
- **Process understanding**: Study vertical water and energy fluxes without spatial complexity
- **Parameter calibration**: Optimize parameters at well-instrumented sites
- **Model development**: Test new parameterizations before scaling up
- **SNOTEL/Weather stations**: Validate snow accumulation and melt processes

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import xarray as xr

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE components
from utils.project.project_manager import ProjectManager
from utils.config.config_utils import ConfigManager
from utils.config.logging_manager import LoggingManager
from utils.geospatial.domain_manager import DomainManager
from utils.data.data_manager import DataManager
from utils.models.model_manager import ModelManager
from utils.evaluation.analysis_manager import AnalysisManager

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Create Point-Scale Configuration

For point-scale modeling, we need to modify our configuration to:
1. Set `SPATIAL_MODE: Point`
2. Create a minimal domain (single point with small buffer)
3. Focus on vertical processes

We'll use a SNOTEL site as our example: Loveland Pass in Colorado.

In [ ]:
# Load template configuration
config_template_path = confluence_path / '0_config_files' / 'config_point_template.yaml'
config_manager = ConfigManager(config_template_path)
config = config_manager.config

# Update directory paths
config['CONFLUENCE_CODE_DIR'] = str(confluence_path)
config['CONFLUENCE_DATA_DIR'] = '/work/comphyd_lab/data/CONFLUENCE_data'

# Modify for point-scale modeling
config['DOMAIN_NAME'] = 'loveland_pass_snotel'
config['EXPERIMENT_ID'] = 'point_scale_tutorial'
config['SPATIAL_MODE'] = 'Point'  # Key setting for point-scale
config['POUR_POINT_COORDS'] = '39.68/-105.88'  # Loveland Pass SNOTEL site

# Use lumped configuration (single unit)
config['DOMAIN_DEFINITION_METHOD'] = 'lumped'
config['DOMAIN_DISCRETIZATION'] = 'lumped'

# Update experiment period for tutorial
config['EXPERIMENT_TIME_START'] = '2020-10-01 00:00'
config['EXPERIMENT_TIME_END'] = '2021-09-30 23:00'

# Set forcing data parameters
config['FORCING_DATASET'] = 'ERA5'
config['FORCING_VARIABLES'] = 'default'  # Will use all available ERA5 variables

# Save point-scale configuration
point_config_path = Path('./point_scale_config.yaml')
with open(point_config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("=== Point-Scale Configuration ===")
print(f"Domain Name: {config['DOMAIN_NAME']}")
print(f"Spatial Mode: {config['SPATIAL_MODE']}")
print(f"Location: {config['POUR_POINT_COORDS']}")
print(f"Period: {config['EXPERIMENT_TIME_START']} to {config['EXPERIMENT_TIME_END']}")
print(f"Forcing Data: {config['FORCING_DATASET']}")

## 3. Visualize Point-Scale vs Spatial Modeling

In [ ]:
# Create conceptual diagram showing point vs spatial modeling
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Point-scale representation
ax1.scatter(0.5, 0.5, s=500, c='red', marker='o', edgecolor='black', linewidth=2)
ax1.text(0.5, 0.7, 'SNOTEL/Flux Tower\n(Single Point)', ha='center', fontsize=14, fontweight='bold')

# Add vertical arrows for fluxes
flux_labels = ['Radiation', 'Precipitation', 'ET', 'Heat Flux']
x_positions = [0.2, 0.4, 0.6, 0.8]
for i, (x, label) in enumerate(zip(x_positions, flux_labels)):
    # Downward arrows
    if label in ['Radiation', 'Precipitation']:
        ax1.arrow(x, 0.45, 0, -0.15, head_width=0.02, head_length=0.03, fc='blue', ec='blue')
        ax1.text(x, 0.25, label, ha='center', fontsize=10, rotation=90)
    # Upward arrows  
    else:
        ax1.arrow(x, 0.55, 0, 0.15, head_width=0.02, head_length=0.03, fc='red', ec='red')
        ax1.text(x, 0.75, label, ha='center', fontsize=10, rotation=90)

ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.set_title('Point-Scale Model', fontsize=16, fontweight='bold')
ax1.axis('off')

# Spatial representation for comparison
from matplotlib.patches import Rectangle
colors = ['lightblue', 'lightgreen', 'wheat', 'lightcoral']
labels = ['Forest', 'Grassland', 'Agriculture', 'Urban']
for i in range(2):
    for j in range(2):
        rect = Rectangle((j*0.4+0.1, i*0.4+0.1), 0.35, 0.35, 
                        facecolor=colors[i*2+j], edgecolor='black')
        ax2.add_patch(rect)
        ax2.text(j*0.4+0.275, i*0.4+0.275, labels[i*2+j], 
                ha='center', va='center', fontsize=10)

ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.set_title('Spatial Model (Multiple Units)', fontsize=16, fontweight='bold')
ax2.axis('off')

plt.suptitle('Point-Scale vs Spatial Modeling Approaches', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Initialize CONFLUENCE for Point-Scale

In [ ]:
# Load point-scale configuration
config_manager = ConfigManager(point_config_path)
config = config_manager.config

# Initialize logger
logging_manager = LoggingManager(config)
logger = logging_manager.logger

# Initialize project manager
project_manager = ProjectManager(config, logger)

print("=== CONFLUENCE Point-Scale Setup ===")
print(f"Project directory: {project_manager.project_dir}")
print(f"Spatial mode: {config['SPATIAL_MODE']}")
print(f"Domain: {config['DOMAIN_NAME']}")

## 5. Step 1: Setup Project Structure

In [ ]:
# Create project structure
print("Creating point-scale project structure...")
project_dir = project_manager.setup_project()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

print("\nNote: For point-scale modeling, we still use the same directory structure")
print("but the spatial extent is minimal (single point with small buffer).")

## 6. Step 2: Create Point Location

In [ ]:
# Create pour point (in this case, our SNOTEL location)
print(f"Creating point location from coordinates: {config['POUR_POINT_COORDS']}")
pour_point_path = project_manager.create_pour_point()

if pour_point_path and pour_point_path.exists():
    print(f"✓ Point location created: {pour_point_path}")
    
    # Quick visualization of the point location
    import geopandas as gpd
    import contextily as cx
    
    gdf = gpd.read_file(pour_point_path)
    gdf_web = gdf.to_crs(epsg=3857)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    gdf_web.plot(ax=ax, color='red', markersize=200, marker='*', 
                 edgecolor='white', linewidth=2, zorder=5)
    
    # Add basemap
    cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery, zoom=12)
    
    # Set extent
    minx, miny, maxx, maxy = gdf_web.total_bounds
    pad = 2000  # 2km padding
    ax.set_xlim(minx - pad, maxx + pad)
    ax.set_ylim(miny - pad, maxy + pad)
    
    ax.set_title('Loveland Pass SNOTEL Site Location', 
                fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    
    # Add text annotation
    lat, lon = gdf.geometry.iloc[0].y, gdf.geometry.iloc[0].x
    ax.text(0.02, 0.98, f'Location: {lat:.4f}°N, {abs(lon):.4f}°W\nElevation: ~3,700 m', 
            transform=ax.transAxes, 
            bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8),
            verticalalignment='top', fontsize=12)
    
    plt.tight_layout()
    plt.show()

## 7. Step 3: Acquire Geospatial Attributes for Point

In [ ]:
# Initialize geospatial data manager
from utils.geospatial.geospatial_utils import GeospatialDataAcquisition
geospatial_manager = GeospatialDataAcquisition(config, logger)

print("Acquiring geospatial attributes for point location...")
print(f"Minimal bounding box: {config.get('BOUNDING_BOX_COORDS')}")
print("Note: CONFLUENCE automatically creates a small buffer for point-scale simulations")

# Acquire attributes (DEM, soil, land cover)
geospatial_manager.acquire_attributes()

# Check downloaded files
attribute_dirs = {
    'DEM': project_dir / 'attributes' / 'elevation' / 'dem',
    'Soil': project_dir / 'attributes' / 'soilclass',
    'Land': project_dir / 'attributes' / 'landclass'
}

print("\nDownloaded attribute files:")
for name, path in attribute_dirs.items():
    if path.exists():
        files = list(path.glob('*.tif')) + list(path.glob('*.tiff'))
        print(f"  {name}: {len(files)} files")

## 8. Step 4: Define Minimal Domain

For point-scale modeling, we create a minimal domain around our point. CONFLUENCE automatically creates a small buffer around the point when `SPATIAL_MODE: Point` is set.

In [ ]:
# Initialize domain manager
domain_manager = DomainManager(config, logger)

print("Defining minimal domain for point-scale simulation...")
print(f"Method: {config['DOMAIN_DEFINITION_METHOD']}")
print(f"Bounding box automatically set to: {config.get('BOUNDING_BOX_COORDS')}")

# Define domain
domain_manager.define_domain()

# Check created files
basin_path = project_dir / 'shapefiles' / 'river_basins'
if basin_path.exists():
    files = list(basin_path.glob('*.shp'))
    print(f"\n✓ Created {len(files)} domain file(s)")
    
    # Load and check the domain
    if files:
        domain_gdf = gpd.read_file(files[0])
        area_km2 = domain_gdf.geometry.area.sum() * 111**2  # Rough conversion
        print(f"Domain area: {area_km2:.2f} km² (minimal)")
        print("This represents a small buffer around the point location")

## 9. Step 5: Create Point-Scale HRU

In [ ]:
# Create HRU (single unit for point-scale)
print(f"Creating single HRU for point-scale simulation...")
print(f"Discretization method: {config['DOMAIN_DISCRETIZATION']}")

# Create HRU
domain_manager.discretize_domain()

# Check the created HRU
hru_path = project_dir / 'shapefiles' / 'catchment'
if hru_path.exists():
    hru_files = list(hru_path.glob('*.shp'))
    print(f"\n✓ Created {len(hru_files)} HRU file(s)")
    
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        print(f"Number of HRUs: {len(hru_gdf)} (should be 1 for point-scale)")
        print(f"HRU ID: {hru_gdf['HRU_ID'].iloc[0]}")
        print("This single HRU represents the point location")

## 10. Step 6: Process Point Observations (if available)

For SNOTEL sites, we often have snow water equivalent (SWE) data. For flux towers, we have additional energy flux measurements.

In [ ]:
print("Processing point-scale observations...")
print("Note: For this tutorial, we'll use synthetic data")
print("In practice, you would load SNOTEL or flux tower data here")

# Create synthetic SNOTEL-like data for demonstration
date_range = pd.date_range(
    start=config['EXPERIMENT_TIME_START'],
    end=config['EXPERIMENT_TIME_END'],
    freq='D'
)

# Generate realistic snow water equivalent pattern
day_of_year = date_range.dayofyear
n_days = len(date_range)

# Create seasonal SWE pattern (peaks in March-April)
swe_seasonal = np.maximum(0, 300 * np.sin(2 * np.pi * (day_of_year - 80) / 365 - np.pi/2))
swe_noise = np.random.normal(0, 10, n_days)
swe = np.maximum(0, swe_seasonal + swe_noise)

# Create snow depth (assuming density of 300 kg/m³)
snow_depth = swe / 300

# Create dataframe
snotel_data = pd.DataFrame({
    'date': date_range,
    'swe_mm': swe,
    'snow_depth_m': snow_depth,
    'temperature_c': -5 + 15 * np.sin(2 * np.pi * day_of_year / 365) + np.random.normal(0, 3, n_days)
})

# Save to observation directory
obs_path = project_dir / 'observations' / 'snow' / 'raw_data'
obs_path.mkdir(parents=True, exist_ok=True)
snotel_data.to_csv(obs_path / f'{config["DOMAIN_NAME"]}_snow_data.csv', index=False)

# Plot the seasonal snow pattern
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.fill_between(snotel_data['date'], 0, snotel_data['swe_mm'], 
                 color='lightblue', alpha=0.7, label='SWE')
ax1.set_ylabel('Snow Water Equivalent (mm)')
ax1.set_title('Synthetic SNOTEL Data for Point-Scale Tutorial', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2.plot(snotel_data['date'], snotel_data['temperature_c'], 
         color='red', linewidth=1.5, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.set_ylabel('Temperature (°C)')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Created synthetic observation data: {obs_path / f'{config['DOMAIN_NAME']}_snow_data.csv'}")

## 11. Step 7: Acquire Forcing Data

In [ ]:
# Initialize data manager
data_manager = DataManager(config, logger)

print(f"Acquiring forcing data for point location...")
print(f"Dataset: {config['FORCING_DATASET']}")
print(f"Period: {config['EXPERIMENT_TIME_START']} to {config['EXPERIMENT_TIME_END']}")
print("\nThe Model Agnostic Framework (MAF) will:")
print("  1. Extract data for the point location")
print("  2. Process meteorological variables")
print("  3. Create model-ready forcing files")

# Run data acquisition through MAF
data_manager.acquire_forcing_data()

# Check outputs
forcing_path = project_dir / 'forcing' / 'easymore-outputs'
if forcing_path.exists():
    files = list(forcing_path.glob('*.nc'))
    print(f"\n✓ Processed {len(files)} forcing files")
    
    if files:
        # Show a sample of the forcing data
        ds = xr.open_dataset(files[0])
        print(f"\nForcing variables: {list(ds.variables)}")
        print(f"Time steps: {len(ds.time)}")
        print(f"Location: {ds.latitude.values[0]:.4f}°N, {ds.longitude.values[0]:.4f}°W")

## 12. Step 8: Model-Agnostic Preprocessing

In [ ]:
# Process forcing data for the point
print("Running model-agnostic preprocessing...")
print("For point-scale simulations:")
print("  - No spatial averaging needed")
print("  - Apply elevation corrections if needed")
print("  - Convert units to model requirements")

# Run preprocessing
data_manager.preprocess_forcing_data()

# Check outputs
processed_forcing_path = project_dir / 'forcing' / 'basin_averaged_data'
if processed_forcing_path.exists():
    files = list(processed_forcing_path.glob('*.nc'))
    print(f"\n✓ Created {len(files)} processed forcing files")
    
    # Quick visualization of forcing data
    if files:
        ds = xr.open_dataset(files[0])
        
        # Plot a week of forcing data
        week_slice = slice('2021-01-01', '2021-01-07')
        fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)
        
        # Temperature
        axes[0].plot(ds.time.sel(time=week_slice), 
                    ds.airtemp.sel(time=week_slice).values - 273.15,
                    color='red', linewidth=2)
        axes[0].set_ylabel('Temperature (°C)')
        axes[0].grid(True, alpha=0.3)
        
        # Precipitation
        axes[1].bar(ds.time.sel(time=week_slice), 
                   ds.pptrate.sel(time=week_slice).values * 3600,
                   width=0.04, color='blue', alpha=0.7)
        axes[1].set_ylabel('Precipitation (mm/hr)')
        axes[1].grid(True, alpha=0.3)
        
        # Solar radiation
        axes[2].plot(ds.time.sel(time=week_sliceds.SWRadAtm.sel(time=week_slice).values,
                    color='orange', linewidth=2)
        axes[2].set_ylabel('Solar Radiation (W/m²)')
        axes[2].set_xlabel('Date')
        axes[2].grid(True, alpha=0.3)
        
        plt.suptitle('Sample Forcing Data for Point Location (1 Week)', fontsize=14)
        plt.tight_layout()
        plt.show()

## 13. Step 9: Model-Specific Preprocessing

In [ ]:
# Initialize model manager
model_manager = ModelManager(config, logger)

print(f"Preparing {config['HYDROLOGICAL_MODEL']} input files for point-scale simulation...")
print("This includes:")
print("  - Model configuration files")
print("  - Parameter files (single HRU)")
print("  - Initial conditions")
print("  - Output control settings")

# Run model-specific preprocessing
model_manager.prepare_model_inputs()

# Check created files
settings_path = project_dir / 'settings' / config['HYDROLOGICAL_MODEL']
if settings_path.exists():
    files = list(settings_path.glob('*'))
    print(f"\n✓ Created {len(files)} model configuration files:")
    for f in files[:10]:  # Show first 10
        print(f"  - {f.name}")
        
    # For point-scale, highlight key configuration aspects
    print("\nPoint-scale specific settings:")
    print("  - Single HRU configuration")
    print("  - No lateral flow between units")
    print("  - Focus on vertical processes")

## 14. Step 10: Run the Point-Scale Model

In [ ]:
# Run the hydrological model
print(f"Running {config['HYDROLOGICAL_MODEL']} for point-scale simulation...")
print("\nPoint-scale models typically:")
print("  - Run faster than distributed models")
print("  - Focus on energy balance and vertical fluxes")
print("  - Are ideal for process-level validation")

# Run the model
model_manager.run_model()

# Check output files
sim_path = project_dir / 'simulations' / config['EXPERIMENT_ID'] / config['HYDROLOGICAL_MODEL']
if sim_path.exists():
    files = list(sim_path.glob('*.nc'))
    print(f"\n✓ Model completed. Created {len(files)} output files.")
    
    # For point-scale, routing is usually not needed
    print("\nNote: Point-scale simulations typically don't require routing")
    print("All outputs are for the single point location")

## 15. Step 11: Analyze Point-Scale Results

In [ ]:
# Initialize analysis manager
analysis_manager = AnalysisManager(config, logger)

print("Analyzing point-scale model results...")
print("Key variables for point-scale analysis:")
print("  - Snow water equivalent (SWE)")
print("  - Soil moisture profiles")
print("  - Energy fluxes (latent, sensible heat)")
print("  - Evapotranspiration")

# Load model output
output_files = list((project_dir / 'simulations' / config['EXPERIMENT_ID'] / 
                    config['HYDROLOGICAL_MODEL']).glob('*.nc'))

if output_files:
    ds_sim = xr.open_dataset(output_files[0])
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(15, 12))
    gs = fig.add_gridspec(4, 2, hspace=0.3)
    
    # Snow water equivalent
    ax1 = fig.add_subplot(gs[0, :])
    if 'scalarSWE' in ds_sim.variables:
        ax1.plot(ds_sim.time, ds_sim.scalarSWE.values, 
                color='blue', linewidth=2, label='Simulated SWE')
        ax1.fill_between(ds_sim.time, 0, ds_sim.scalarSWE.values, 
                        color='lightblue', alpha=0.5)
    ax1.set_ylabel('SWE (mm)')
    ax1.set_title('Snow Water Equivalent', fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Temperature and precipitation
    ax2 = fig.add_subplot(gs[1, 0])
    if 'scalarRainPlusMelt' in ds_sim.variables:
        ax2.bar(ds_sim.time, ds_sim.scalarRainPlusMelt.values * 3600, 
               width=0.04, color='blue', alpha=0.7, label='Rain+Melt')
    ax2.set_ylabel('Water Input (mm/hr)')
    ax2.set_title('Precipitation and Melt', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Soil moisture
    ax3 = fig.add_subplot(gs[1, 1])
    if 'scalarSoilMoisture' in ds_sim.variables:
        ax3.plot(ds_sim.time, ds_sim.scalarSoilMoisture.values, 
                color='brown', linewidth=2)
    ax3.set_ylabel('Soil Moisture (-)')
    ax3.set_title('Soil Moisture Content', fontsize=12)
    ax3.grid(True, alpha=0.3)
    
    # Energy fluxes
    ax4 = fig.add_subplot(gs[2, :])
    if 'scalarLatHeatTotal' in ds_sim.variables:
        ax4.plot(ds_sim.time, ds_sim.scalarLatHeatTotal.values, 
                color='green', linewidth=1.5, label='Latent Heat', alpha=0.7)
    if 'scalarSenHeatTotal' in ds_sim.variables:
        ax4.plot(ds_sim.time, ds_sim.scalarSenHeatTotal.values, 
                color='red', linewidth=1.5, label='Sensible Heat', alpha=0.7)
    ax4.set_ylabel('Energy Flux (W/m²)')
    ax4.set_title('Surface Energy Fluxes', fontsize=12)
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    # ET components
    ax5 = fig.add_subplot(gs[3, :])
    if 'scalarActualET' in ds_sim.variables:
        ax5.plot(ds_sim.time, ds_sim.scalarActualET.values * 3600, 
                color='purple', linewidth=2, label='Total ET')
    ax5.set_ylabel('ET (mm/hr)')
    ax5.set_xlabel('Date')
    ax5.set_title('Evapotranspiration', fontsize=12)
    ax5.grid(True, alpha=0.3)
    ax5.legend()
    
    plt.suptitle(f'Point-Scale Model Results: {config["DOMAIN_NAME"]}', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    plot_path = project_dir / 'plots' / 'results'
    plot_path.mkdir(parents=True, exist_ok=True)
    fig.savefig(plot_path / 'point_scale_results.png', dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved results plot to: {plot_path / 'point_scale_results.png'}")

## 16. Compare with Observations (if available)

In [ ]:
# Load observation data
obs_file = project_dir / 'observations' / 'snow' / 'raw_data' / f'{config["DOMAIN_NAME"]}_snow_data.csv'
if obs_file.exists():
    obs_data = pd.read_csv(obs_file)
    obs_data['date'] = pd.to_datetime(obs_data['date'])
    
    # Create comparison plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot observations
    ax.plot(obs_data['date'], obs_data['swe_mm'], 
            color='black', linewidth=2, label='Observed SWE', alpha=0.7)
    
    # Plot simulations if available
    if 'scalarSWE' in ds_sim.variables:
        ax.plot(pd.to_datetime(ds_sim.time.values), ds_sim.scalarSWE.values, 
                color='red', linewidth=2, label='Simulated SWE', alpha=0.7)
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Snow Water Equivalent (mm)', fontsize=12)
    ax.set_title('Point-Scale Model Validation: SWE Comparison', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate basic metrics if both exist
    if 'scalarSWE' in ds_sim.variables:
        # Align time series
        sim_df = pd.DataFrame({'date': pd.to_datetime(ds_sim.time.values),
                              'sim_swe': ds_sim.scalarSWE.values})
        merged = pd.merge(obs_data, sim_df, on='date', how='inner')
        
        if len(merged) > 0:
            rmse = np.sqrt(np.mean((merged['swe_mm'] - merged['sim_swe'])**2))
            bias = np.mean(merged['sim_swe'] - merged['swe_mm'])
            corr = merged[['swe_mm', 'sim_swe']].corr().iloc[0, 1]
            
            print("\nModel Performance Metrics:")
            print(f"RMSE: {rmse:.2f} mm")
            print(f"Bias: {bias:.2f} mm")
            print(f"Correlation: {corr:.3f}")

## 17. Summary: Point-Scale Modeling Insights

### What We Accomplished:

1. **Set up point-scale simulation** for a SNOTEL site
2. **Minimal spatial domain** with single HRU
3. **Focused on vertical processes** without lateral flow
4. **Analyzed key variables** relevant to point observations

### Key Differences from Spatial Models:

- **No spatial heterogeneity**: Single point representation
- **No routing**: All processes occur at one location
- **Direct comparison**: Easy validation with point observations
- **Computational efficiency**: Fast execution for testing

### Applications of Point-Scale Modeling:

1. **Parameter calibration**: Optimize parameters at well-observed sites
2. **Process validation**: Test model physics against observations
3. **Model development**: Develop new parameterizations
4. **Sensitivity analysis**: Understand parameter influence

### Next Steps:

1. **Scale up to lumped basin**: Use calibrated parameters
2. **Compare multiple sites**: Test model transferability
3. **Ensemble simulations**: Explore parameter uncertainty
4. **Process experiments**: Test different model configurations

In [ ]:
# Print final summary
print("=== Point-Scale Workflow Complete ===\n")
print(f"Project: {config['DOMAIN_NAME']}")
print(f"Location: {config['POUR_POINT_COORDS']}")
print(f"Model: {config['HYDROLOGICAL_MODEL']}")
print(f"Period: {config['EXPERIMENT_TIME_START']} to {config['EXPERIMENT_TIME_END']}")

print("\nKey characteristics of point-scale modeling:")
print("  ✓ Single spatial unit (1 HRU)")
print("  ✓ Focus on vertical processes")
print("  ✓ Direct comparison with observations")
print("  ✓ Computationally efficient")
print("  ✓ Ideal for process understanding")

print("\nKey outputs:")
print(f"  - Configuration: {point_config_path}")
print(f"  - Model results: {project_dir}/simulations/{config['EXPERIMENT_ID']}/")
print(f"  - Visualizations: {project_dir}/plots/results/")
print(f"  - Forcing data: {project_dir}/forcing/basin_averaged_data/")